# Multi-Gaussian example

As an introductory example we can use astroABC to find the posterior distribution for some Gaussian distributed data. Although in this case we already know the likelihood this example is to illustrate how to call astroABC and provide user-defined metrics.


In [25]:
# start by importing astroabc and numpy
import numpy as np
import astroabc

We need to provide:

- a dataset
- a forwards simulating model for the data
- a method defining the distance metric.

For this example we generate a dataset of a 1000 draws from a 2D multi-Gaussian where the true means are e.g

$\mu_{1,true} = 0.037579, \, \mu_{2, true}=0.573537$

and we have fixed the covariance matrix to be  a diagonal matrix with $\sigma_1^2 = \sigma_2^2 = 0.05$. 
We can do this using an inbuilt model class in astroabc.

In [26]:
#make the fake data
means= np.array([0.037579, 0.573537])
data = astroabc.Model("normal",1000).make_mock(means)

In this example the make_mock method also provides a forwards simulating model for the data. 

In [27]:
#define a method for simulating the data given input parameters
model_sim = astroabc.Model("normal",10000).make_mock

Next we define a distance metric method. In this example instead of using all of the data (all 1000 draws from a 2D Gaussian) we use the means of the data as a summary statistic and our distance metric is the sum over the difference in the means for the 2D Gaussian 

In [28]:
def dist_metric(d,x):
    return np.sum(np.abs(np.mean(x,axis=0) - np.mean(d,axis=0)))

Next we specify priors for each of out parameters we want to vary in the sampler. This is done by specifying a list of tuples. The zeroth element in each tuple should be a string specifying the prior for this parameter and the first element should be a list of the hyperparameters needed for this prior.
e.g. below we use a normal distribution with mean  = 0 and variance 0.5 for the first parameter and a uniform distribution from 0.1 - 0.9 for the second parameter.

In [29]:
priors =  [('normal', [0.0,0.5]), ('uniform', [0.1, 0.9])]

Next we need to set some keywords for astroABC. This can be done by creating a dictionary of inputs which are passed to the sampler. Many of these entries have defaults and do not need to be specified explicitly.
Only the name of the distance metric method needs to be explicity provided as a keyword.
The full set of keywords are given in the doc string of the class. Some examples are

- 'tol_type': which specifies how the decreasing tolerance levels are defined. "exp","lin", "log" and "const" are options. (default = 'exp')

- "verbose": level of verbosity, 0 = no printout to screen, 1 = print to screen  (default = 0)

- 'adapt_t': Boolean True/False for adaptive threshold setting (default = False)

- 'threshold': qth quantile used in adaptive threshold setting (default = 75)

- 'pert_kernel': 0 =weighted covariance, 1= Filippi, 2 = TVZ, 3= Leodoit_Wolf (default = 0)

- 'dfunc':method for calculating the distance metric

- 'from_restart': Boolean True/False

- 'restart': string name of restart file

- 'outfile':string specifying name of output file (default = abc_out.txt)

- 'mpi': Boolean True/False (default = False)

- 'mp':Boolean True/False (default = False)

- 'num_proc':number of threads for mp setting (default = None)

Please see the doc strings of the astroABC sampler for details on each of these settings.

In [30]:
prop={'dfunc':dist_metric, 'outfile':"gaussian_example.txt", 'verbose':1, 'adapt_t': True}

Now we are ready to create an instance of our sampler. 
To do this we just need to specify the following to the astroabc.ABC_class

astroabc.ABC_class(number of parameters,number of particles,data,tolerance levels,number of iterations,priors,prop)

To begin let's run in serial using 100 particles for 30 iterations with default tolerance levels of a maximum threshold of 0.7 and  a minimum threshold of 0.05:

In [19]:
sampler = astroabc.ABC_class(2,100,data,[0.7,0.01],30,priors,**prop)

	 	
	 ########################     astroABC     ########################	
	 	
	 Npart=100 	 numt=30 	 tol=[0.70,0.01] exp
	 Priors= [('normal', [0.0, 0.5]), ('uniform', [0.1, 0.9])]


Then we simply begin sampling on our data...

In [20]:
sampler.sample(model_sim)

	 Running sampler...	 
	 Step: 0 	 tol: 0.7 	 Params: [0.036105913495735158, 0.51350995565281299]
	 Step: 1 	 tol: 0.552481476079 	 Params: [0.030735942934779738, 0.54171177742180665]
	 Step: 2 	 tol: 0.461035151041 	 Params: [0.023759200102021559, 0.54588144274467143]
	 Step: 3 	 tol: 0.381780740958 	 Params: [0.034090742420534982, 0.55684416521988855]
	 Step: 4 	 tol: 0.320225489753 	 Params: [0.0204295220246925, 0.56418648596605459]
	 Step: 5 	 tol: 0.266195049164 	 Params: [0.017333893623035012, 0.55747736326596542]
	 Step: 6 	 tol: 0.217438084252 	 Params: [0.031955812034894795, 0.58414395465955671]
	 Step: 7 	 tol: 0.182561095516 	 Params: [0.032304787734206267, 0.56821745786451838]
	 Step: 8 	 tol: 0.152839596245 	 Params: [0.034423297530541304, 0.5581542550416958]
	 Step: 9 	 tol: 0.128215304182 	 Params: [0.039588114760285439, 0.5704658191732489]
	 Step: 10 	 tol: 0.111752741786 	 Params: [0.037288405074433609, 0.56809131436704385]
	 Step: 11 	 tol: 0.0907983794615 	 Params: [

The output above shows the estimated means of the 2D Gaussian averaged over all 100 particles at each iteration, together with the tolerance level. Note above that the sampling may end before 20 iterations if the minimum tolerance level is reached first.
Recall that the true parameter values are $\mu_{1,true} = 0.037579, \, \mu_{2, true}=0.573537$

# Running astroABC in parallel

To re-do the above analysis in parallel there are two options.
For running on a cluster it is best to use the mpi option and at run time specify the number of processors using

mpirun -np # 

To use the mpi option in astroABC simply set 'mpi': True in the keywords for the class. 

In [22]:
prop={'dfunc':dist_metric, 'outfile':"gaussian_example.txt", 'verbose':1, 'adapt_t': True, 'mpi': True}

You can then run the sample script in the examples folder to run the gaussian example on e.g. 16 processors using

$> mpirun -np 16 python gaussian.py gaussian_params.ini

For smaller jobs the Python multiprocessing option is available which can spawn multiple processes but which are still bound within a single node.
To use the mp option in astroABC simply set 'mp': True in the keywords for the class.
An additional option is to specify the number of threads using 'num_proc':number of threads for mp setting.
If none is specified then all available threads are used.

In [24]:
#to run on 4 threads
prop={'dfunc':dist_metric, 'outfile':"gaussian_example.txt", 'verbose':1, 'adapt_t': True, 'mp': True, 'num_proc':4}